In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import sys
sys.path.append("/home/msst/repo/Quantization")
from qlib._modeling.modeling_llama import LlamaForCausalLM
from qlib import evaluate, QATDataset

import torch
from transformers import AutoTokenizer

from utils import init_quant_model_base, init_quant_model_hessian, QuantLinear

/home/msst/Utils/miniconda3/envs/qenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FP

path_to_model = "/media/msst/ssd_storage1/ml/llm/pretrained_models/Llama2-7B"
tokenizer = AutoTokenizer.from_pretrained(path_to_model)
model_fp = LlamaForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 112.62it/s]


In [3]:
BITS = 2
GROUP_SIZE = 64

model_q = LlamaForCausalLM.from_pretrained(
    path_to_model,
    device_map="cpu",
    dtype="auto"
)

def wrap_model(current_module, prefix=''):
    for module_name, module in current_module.named_children():
        full_name = f"{prefix}.{module_name}" if prefix else module_name
        
        if "proj" in module_name:
            weight_shape = module.weight.data.shape
            setattr(current_module, module_name, QuantLinear(weight_shape, GROUP_SIZE, BITS))
        else:
            wrap_model(module, full_name)

wrap_model(model_q)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 110.63it/s]


In [ ]:
path = f"/home/msst/repo/Quantization/nb/adaptive_rounding/init_w{BITS}gs{GROUP_SIZE}.pth"

# init_quant_model_base(model_q, model_fp)
# torch.save(model_q.state_dict(), path)
model_q.load_state_dict(torch.load(path))

tensor(0.4820) model.layers.0.self_attn.q_proj
tensor(0.4102) model.layers.0.self_attn.k_proj
tensor(0.2148) model.layers.0.self_attn.v_proj
tensor(0.2834) model.layers.0.self_attn.o_proj
tensor(0.2047) model.layers.0.mlp.gate_proj
tensor(0.2044) model.layers.0.mlp.up_proj
tensor(0.2075) model.layers.0.mlp.down_proj
tensor(0.2335) model.layers.1.self_attn.q_proj
tensor(0.2376) model.layers.1.self_attn.k_proj
tensor(0.2192) model.layers.1.self_attn.v_proj
tensor(0.2485) model.layers.1.self_attn.o_proj
tensor(0.2049) model.layers.1.mlp.gate_proj
tensor(0.2046) model.layers.1.mlp.up_proj
tensor(0.2054) model.layers.1.mlp.down_proj
tensor(0.2113) model.layers.2.self_attn.q_proj
tensor(0.2099) model.layers.2.self_attn.k_proj
tensor(0.2057) model.layers.2.self_attn.v_proj
tensor(0.2063) model.layers.2.self_attn.o_proj
tensor(0.2037) model.layers.2.mlp.gate_proj
tensor(0.2040) model.layers.2.mlp.up_proj
tensor(0.2037) model.layers.2.mlp.down_proj
tensor(0.2101) model.layers.3.self_attn.q_proj

<All keys matched successfully>

In [ ]:
config = {
    "dataset_name" : "slim_pajama",
    "split": "train[:250]",
    "seq_length": 4096,
    "batch_size": 8,
    "random_seed": 'no_rand'
}

dataloader = QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()

init_quant_model_hessian(
    model_q, 
    model_fp, 
    dataloader, 
    advanced=True, 
    # init_blocks=3, 
    use_cholesky=True
)

  0%|          | 0/32 [00:00<?, ?it/s]/home/msst/Utils/miniconda3/envs/qenv/lib/python3.13/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
/home/msst/Utils/miniconda3/envs/qenv/lib/python3.13/site-packages/torch/_inductor/compile_fx.py:312: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better per

312.4642028808594 -> 25.386293411254883
384.5408020019531 -> 27.397750854492188
226.7843017578125 -> 9.038747787475586
2.4171228408813477 -> 0.43604612350463867
434.210205078125 -> 240.4371795654297
433.3564453125 -> 230.09152221679688
1.9809553623199463 -> 1.0955758094787598


  3%|▎         | 1/32 [01:26<44:53, 86.90s/it]

5276.001953125 -> 991.803466796875
5321.421875 -> 1011.8199462890625
1067.2955322265625 -> 110.42594909667969
12.065034866333008 -> 6.955783843994141
1827.060546875 -> 1414.498779296875
1490.5360107421875 -> 1100.546142578125
7141.8779296875 -> 183.28485107421875


  6%|▋         | 2/32 [02:49<42:07, 84.24s/it]

12063.65625 -> 4457.0283203125
12613.712890625 -> 4867.1748046875
3904.09326171875 -> 1170.191162109375
45.68397521972656 -> 16.34051513671875
3797.11865234375 -> 3130.78076171875
3159.1015625 -> 2495.4951171875
23.85717010498047 -> 20.610610961914062


  9%|▉         | 3/32 [04:11<40:27, 83.71s/it]


In [6]:
config = {
    "dataset_name" : "wiki",
    "split": "test",
    "seq_length": 2048,
    "batch_size": 1,
    "random_seed": 'no_rand'
}

dataloader = QATDataset(
    config=config,
    tokenizer=tokenizer
).get_dataloader()

model_q = model_q.cuda()
# model_q = model_q.cuda()

with torch.no_grad():
    res = evaluate(model_q, dataloader, print_times=10)
    print(res)

  0%|          | 0/166 [00:00<?, ?it/s]W0106 13:57:55.692000 31607 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] torch._dynamo hit config.recompile_limit (8)
W0106 13:57:55.692000 31607 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8]    function: 'reconstruct_weight' (/home/msst/repo/Quantization/nb/adaptive_rounding/utils.py:32)
W0106 13:57:55.692000 31607 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8]    last reason: 0/7: tensor 'self._buffers['compressed_weight']' size mismatch at index 0. expected 11008, actual 4096
W0106 13:57:55.692000 31607 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0106 13:57:55.692000 31607 site-packages/torch/_dynamo/convert_frame.py:1358] [0/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html
 10%|█         | 17/166 [00:09<01:38,  1.51it/s]

155.07044677221742


 20%|█▉        | 33/166 [00:18<01:28,  1.51it/s]

172.97263319046485


 30%|██▉       | 49/166 [00:27<01:17,  1.51it/s]

143.01810211709636


 39%|███▊      | 64/166 [00:36<00:58,  1.74it/s]


KeyboardInterrupt: 

In [ ]:
# setting: w3gs128
# wiki ppl seqlen=2048

# 8.76 - base
# 6.41 - advanced=False, init_blocks=3 
# 6.39 - advanced=True, init_blocks=3
# 6.42 - advanced=True, init_blocks=3, use_cholesky=True

In [ ]:
# setting: w3gs128
# wiki ppl seqlen=2048

#  - base
#  - advanced=False, init_blocks=3 
#  - advanced=True, init_blocks=3
#  - advanced=True, init_blocks=3, use_cholesky=True